In [1]:
{-# LANGUAGE FlexibleInstances #-}
{-# LANGUAGE MultiParamTypeClasses #-}
{-# LANGUAGE TypeFamilies #-}
{-# LANGUAGE TypeOperators #-}

In [2]:
import Control.Monad.State

## Types

### Type Constructors

Unary types include entities $e$, truth values $t$, and discourse contexts
$\sigma$ (modeled here as lists).
Constructed types take one of the following forms, where $\alpha$ and
$\beta$ are any two types:

* $\alpha \to \beta$, the type of a function from $\alpha$ to $\beta$.
* $\left\{\alpha\right\}$, the type of a set of $\alpha$ objects.
* $\alpha \ast \beta$, the type of an $\alpha$ object paired with
  a $\beta$ object, in that order.

### Type Abbreviations
 
To keep type descriptions readable, I use the following type synonyms: $\alpha$


* $\mathbb{D}\,{\alpha} \equiv \sigma\to \left\{\alpha \ast \sigma\right\}$,
  the type of updates corresponding to constituents of type $\alpha$.

* $\mathbb{K}\,{\rho}\,{\omicron}\,{\alpha} \equiv \left(\alpha\to \omicron\right)\to
  \rho$, the type of generalized quantifiers with base type
  $\alpha$.
* $\mathbb{F}\,{\alpha} \equiv \mathbb{D}\,{\alpha}\to \mathbb{D}\,{\alpha}$,
  the type of filters on updates.

In [3]:
type E = String
type T = Bool

type Stack = [E]

type D = StateT Stack []
type F a = D a -> D a
data K r o a = Tower {runTower :: (a -> o) -> r}

### Modes of combination

#### Recursive forward application

$
\begin{aligned}
m \,\big/\!\!\big/\, n
&=
\begin{cases}
  m\,n \quad 
  \textbf{if}\ m\, {\colon\!\colon}\, \alpha\to\beta,\ n\, {\colon\!\colon}\, \alpha \\
  \lambda k .\, m\,{\left(\lambda f .\, n\,{\left(\lambda x .\, k\,{\left(f \,\big/\!\!\big/\, x\right)}\right)}\right)} \quad
  \textbf{otherwise}
\end{cases}
\end{aligned}
$

In [4]:
type family f :/: g where
  (a -> b) :/: a = b
  (K c d f) :/: (K d e x) = K c e (f :/: x)

infixr 9 ~/~
class AppR f g where
  (~/~) :: f -> g -> f :/: g
instance AppR (a -> b) a where
  f ~/~ x = f x
instance (d ~ d', AppR in1 in2) => AppR (K c d in1) (K d' e in2) where
  mf ~/~ mx = Tower $ \k -> runTower mf (\f -> runTower mx (\x -> k (f ~/~ x)))

`:/:` is a type-level function that takes two types, `f` and `g`, and returns
the type that would result from combining something of type `f` on the left with something
of type `g` on the right

And `~/~` is the polymorphic function that "applies" a function-y thing on the left
to an argument-y thing; it is // above

#### Recursive backward application

$
\begin{aligned}
m \,\backslash\!\!\backslash\, n
&=
\begin{cases}
  n\,m \quad
  \textbf{if}\ n\, {\colon\!\colon}\, \alpha\to\beta,\ m\, {\colon\!\colon}\, \alpha \\
  \lambda k .\, m\,{\left(\lambda x .\, n\,{\left(\lambda f .\, k\,{\left(x \,\backslash\!\!\backslash\, f\right)}\right)}\right)} \quad 
  \textbf{otherwise}
\end{cases}
\end{aligned}
$

In [5]:
type family f :\: g where
  a :\: (a -> b) = b
  (K c d x) :\: (K d e f) = K c e (x :\: f)

infixr 9 ~\~
class AppL f g where
  (~\~) :: f -> g -> f :\: g
instance AppL a (a -> b) where
  x ~\~ f = f x
instance (d ~ d', AppL in1 in2) => AppL (K c d in1) (K d' e in2) where
  mx ~\~ mf = Tower $ \k -> runTower mx (\x -> runTower mf (\f -> k (x ~\~ f)))

#### Recursive predicate modification

$
\begin{aligned}
m \,\,\big|\hspace{0.5pt}\big|\,\, n
&=
\begin{cases}
  \lambda x .\, m\,x \land n\,x \quad
  \textbf{if}\ m\, {\colon\!\colon}\, \alpha\to t,\ n\, {\colon\!\colon}\, \alpha\to t \\
  \lambda k .\, m\,{\left(\lambda p .\, n\,{\left(\lambda q .\, k\,{\left(p \,\,\big|\hspace{0.5pt}\big|\,\, q\right)}\right)}\right)} \quad 
  \textbf{otherwise}
\end{cases}
\end{aligned}
$

In [6]:
type family f :|: g where
  (a -> T) :|: (a -> T) = a -> T
  (K c d f) :|: (K d e g) = K c e (f :|: g)

infixr 9 ~|~
class AppC f g where
  (~|~) :: f -> g -> f :|: g
instance AppC (a -> T) (a -> T) where
  f ~|~ g = \x -> f x && g x
instance (d ~ d', AppC in1 in2) => AppC (K c d in1) (K d' e in2) where
  mf ~|~ mg = Tower $ \k -> runTower mf (\f -> runTower mg (\g -> k (f ~|~ g)))

To help the type-checker resolve some of the rampant polymorphism, the following
(non-recursive) special cases of $\,\big/\!\!\big/\,$ and $\,\backslash\!\!\backslash\,$ are defined

In [7]:
mf </> mx = Tower $ \k -> runTower mf (\f -> runTower mx (\x -> k (f x)))
mf <//> mx = Tower $ \k -> runTower mf (\f -> runTower mx (\x -> k (f </> x)))
mx <\> mf = Tower $ \k -> runTower mx (\x -> runTower mf (\f -> k (f x)))
mx <\\> mf = Tower $ \k -> runTower mx (\x -> runTower mf (\f -> k (x <\> f)))

### Evaluation

$
\begin{aligned}
m^{\Downarrow}
&=
\begin{cases}
  m\,\eta \quad 
  \textbf{if}\ m\, {\colon\!\colon}\, \mathbb{K}\,{\rho}\,{\left(\mathbb{D}\,{\alpha}\right)}\,{\alpha}\\
  m\,{\left(\lambda n.\, n^\Downarrow\right)} \quad
  \textbf{otherwise}
\end{cases}
\end{aligned}
$

In [8]:
class Low r o a where
  lowr :: K r o a -> r
instance Low r (D a) a where
  lowr t = runTower t return
instance (Low r' o' a, o ~ r') => Low r o (K r' o' a) where
  lowr t = runTower t lowr

Note that any given application of $\Downarrow$ is equivalent to one of

$
\begin{gathered}
{\left(\eta\,\eta\right)}^{\star}\\
{\left(\eta\,{\left(\eta\,\eta\right)}^{\star}\right)}^{\star}\\
{\left(\eta\,{\left(\eta\,{\left(\eta\,\eta\right)}^{\star}\right)}^{\star}\right)}^{\star}\\
\dots
\end{gathered}
$

### Other convenience functions

$
\begin{aligned}
x^{\uparrow}
&=
{\left(\eta\,x\right)}^{\star}
&
x^{\uparrow\!\uparrow}
&=
{\left(\eta\,{\left(\eta\,x\right)}^{\star}\right)}^{\star}
\\
&=
\lambda k.\, k\,x
&
&=
\lambda \gamma.\, \gamma\,{\left(\lambda k.\, k\,x\right)}
\end{aligned}
$

In [9]:
u :: a -> K (D r) (D r) a
u x = Tower (return x >>=)

uu :: a -> K (D r') (D r') (K (D r) (D r) a)
uu = u . u

$
\begin{aligned}
m^{\overline{\Downarrow}}
&=
\lambda k.\, m\,{\left(\lambda n.\, k\,n^{\Downarrow}\right)}
\end{aligned}
$

In [10]:
ilowr :: Low r o a => K (D r') o' (K r o a) -> K (D r') o' r
ilowr = (u lowr </>)

$
\begin{aligned}
m^{\downharpoonleft\!\upharpoonright}
&=
{\left(m^{\Downarrow}\right)}^{\star}
\end{aligned}
$

In [11]:
reset :: (r ~ D b, Low r o a) => K r o a -> K (D r') (D r') b
reset t = Tower (lowr t >>=)